In [1]:
import requests
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# Create a custom requests session
session = requests.Session()

# Set custom user-agent to comply with Wikipedia's policy
user_agent = "MichaelPhipps/1.0 (mwp897@gmail.com)  Python/wikipedia-api"
# Set custom user-agent to comply with Wikipedia's policy

# Initialize the Wikipedia API for English language with the user-agent
wiki_wiki = wikipediaapi.Wikipedia(
    'en',
    headers={'User-Agent': user_agent}
)


ModuleNotFoundError: No module named 'wikipediaapi'

In [27]:
import wikipedia

def search_wikipedia(query):
    # Use wikipedia package to search for articles
    search_results = wikipedia.search(query)
    if len(search_results) == 0:
        print(f"No results found for query: {query}")
        return None
    print(f"Top search result: {search_results[0]}")
    print(f"Full search results: {search_results}")
    return search_results[0]  # Return the first (most relevant) result

def get_wikipedia_article(query):
    # First, try to find the relevant article title using search
    title = search_wikipedia(query)
    if not title:
        return None
    
    # Now fetch the Wikipedia article using the retrieved title
    page = wiki_wiki.page(title)
    
    if not page.exists():
        print(f"The article '{title}' does not exist on Wikipedia.")
        return None
    
    # Get the summary and full text of the article
    summary = page.summary
    full_text = page.text
    url = page.fullurl
    
    return {
        'title': page.title,
        'summary': summary,
        'full_text': full_text,
        'url': url
    }

In [3]:
# Get article for "School of Athens"
school_of_athens = get_wikipedia_article("School of Athens")
print(f"Title: {school_of_athens['title']}\n")
print(f"Summary:\n{school_of_athens['summary']}\n")
print(f"Full URL: {school_of_athens['url']}\n")

# Get article for "CSS Flex"
css_flex = get_wikipedia_article("css flex")
print(f"Title: {css_flex['title']}\n")
print(f"Summary:\n{css_flex['summary']}\n")
print(f"Full URL: {css_flex['url']}\n")

Top search result: The School of Athens
Full search results: ['The School of Athens', 'British School at Athens', 'Athens Academy (school)', 'Athens High School (Ohio)', 'French School at Athens', 'Athens', 'International School of Athens', 'Athens High School (Alabama)', 'Athens City Schools', 'Barrow County Schools']
Title: The School of Athens

Summary:
The School of Athens (Italian: Scuola di Atene) is a fresco by the Italian Renaissance artist Raphael. It was painted between 1509 and 1511 as part of a commission by Pope Julius II to decorate the rooms now called the Stanze di Raffaello in the Apostolic Palace in Vatican City. 
The fresco depicts a congregation of ancient philosophers, mathematicians, and scientists, with Plato and Aristotle featured in the center. The identities of most figures are ambiguous or discernable only through subtle details or allusions; among those commonly identified are Socrates, Pythagoras, Archimedes, Heraclitus, Averroes, and Zarathustra. Additiona

In [4]:
# Get article for user query
query = get_wikipedia_article("President of the United States")
print(f"Title: {query['title']}\n")
print(f"Summary:\n{query['summary']}\n")
print(f"Full URL: {query['url']}\n")

Top search result: President of the United States
Full search results: ['President of the United States', 'List of presidents of the United States', 'Vice President of the United States', 'List of presidents of the United States by age', 'Historical rankings of presidents of the United States', 'Powers of the president of the United States', 'List of vice presidents of the United States', 'President pro tempore of the United States Senate', 'Thomas Jefferson', 'Grover Cleveland']
Title: President of the United States

Summary:
The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasi

In [28]:

from pydantic import BaseModel
from llama_index.llms.openai.utils import to_openai_tool
from llama_index.core.tools import FunctionTool


class Song(BaseModel):
    """A song with name and artist"""

    name: str
    artist: str


def generate_song(name: str, artist: str) -> Song:
    """Generates a song with provided name and artist."""
    return Song(name=name, artist=artist)


tool = FunctionTool.from_defaults(fn=generate_song)

NameError: name 'generate_song' is not defined

In [10]:
import wikipedia
from pydantic import BaseModel
# Disable caching in the wikipedia library
wikipedia.set_rate_limiting(False)

class WikiSearchResult(BaseModel):
    title: str
    url: str

class WikiArticle(BaseModel):
    title: str
    content: str
    url: str

def wikipedia_similar_articles(query: str) -> list[WikiSearchResult]:
    """Search Wikipedia for articles similar to the given query and return titles and URLs."""
    search_results = wikipedia.search(query, results=15)
    result_list = []
    for result in search_results:
        try:
            page = wikipedia.page(result)
            result_list.append(WikiSearchResult(title=page.title, url=page.url))
        except wikipedia.exceptions.DisambiguationError as e:
            # Handle disambiguation pages by logging or ignoring
            print(f"Disambiguation page: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"PageError: {result}")
    return result_list



def wikipedia_full_article(query: str) -> WikiArticle:
    """Fetch the full Wikipedia article for the given query."""
    try:
        page = wikipedia.page(query)
        return WikiArticle(title=page.title, content=page.content, url=page.url)
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        print(f"Disambiguation page: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"PageError: {query}")
    return None


In [12]:
print(wikipedia_full_article("Donald Trump 2024 presidential campaign"))

PageError: Donald Trump 2024 presidential campaign
None


In [17]:
print(wikipedia_similar_articles("Attempted assassination of Donald Trump"))

PageError: Thomas Matthew Crooks
PageError: Kimberly Cheatle
PageError: The Boys season 4
[WikiSearchResult(title='Attempted assassination of Donald Trump', url='https://en.wikipedia.org/wiki/Attempted_assassination_of_Donald_Trump'), WikiSearchResult(title='Donald Trump raised-fist photographs', url='https://en.wikipedia.org/wiki/Donald_Trump_raised-fist_photographs'), WikiSearchResult(title='2016 Donald Trump Las Vegas rally incident', url='https://en.wikipedia.org/wiki/2016_Donald_Trump_Las_Vegas_rally_incident'), WikiSearchResult(title='Security incidents involving Donald Trump', url='https://en.wikipedia.org/wiki/Security_incidents_involving_Donald_Trump'), WikiSearchResult(title='Donald Trump', url='https://en.wikipedia.org/wiki/Donald_Trump'), WikiSearchResult(title='Attempted assassination of Donald Trump', url='https://en.wikipedia.org/wiki/Attempted_assassination_of_Donald_Trump'), WikiSearchResult(title='Meridian, Pennsylvania', url='https://en.wikipedia.org/wiki/Meridian,_P

Search returns PageErrors for many newly edited pages (maybe a caching issue?). Means our agent will struggle on queries related to current events.


Also interesting that "Attempted assassination of Donald Trump" returns the shooter's name. Means search goes beyond TF-IDF/BM-25. Must incorporate tags/aliases or potentially semantic search.

In [29]:
import wikipedia
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

class WikiSearchResult(BaseModel):
    title: str
    url: str

class WikiArticle(BaseModel):
    title: str
    content: str
    url: str

def wikipedia_similar_articles(query: str) -> list[WikiSearchResult]:
    """Search Wikipedia for articles similar to the given query and return titles and URLs."""
    search_results = wikipedia.search(query, results=5)
    result_list = []
    for result in search_results:
        try:
            page = wikipedia.page(result)
            result_list.append(WikiSearchResult(title=page.title, url=page.url))
        except wikipedia.exceptions.DisambiguationError as e:
            # Handle disambiguation pages by logging or ignoring
            print(f"Disambiguation page: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"PageError: {result}")
    return result_list



def wikipedia_full_article(query: str) -> WikiArticle:
    """Fetch the full Wikipedia article for the given query."""
    try:
        page = wikipedia.page(query)
        return WikiArticle(title=page.title, content=page.content, url=page.url)
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        print(f"Disambiguation page: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"PageError: {query}")
    return None

# Wrap these functions in a tool
similar_articles_tool = FunctionTool.from_defaults(fn=wikipedia_similar_articles)
full_article_tool = FunctionTool.from_defaults(fn=wikipedia_full_article)


# Now use the LLM to call this function
llm = OpenAI(model="gpt-3.5-turbo", strict=True)

# query = "Tell me a joke"
query = "Tell me about the Eiffel Tower"
# query = "Give me articles similar to 'Philosophy'"
# Call predict_and_call with both tools
response = llm.predict_and_call(
    [similar_articles_tool, full_article_tool],
    query 
)

# Process the response based on which tool was called
if response.sources:  # Check if the response contains sources
    for source in response.sources:
        # Print the tool name that was used
        print(f"Tool used: {source.tool_name}")

        # Each source should contain the ToolOutput with the actual results
        tool_output = source.raw_output

        # Check which tool was used and process the result accordingly
        if isinstance(tool_output, list) and all(isinstance(item, WikiSearchResult) for item in tool_output):
            # This means the similar_articles_tool was used
            print("Similar Articles:")
            for result in tool_output:
                print(f"Title: {result.title}, URL: {result.url}")
        
        elif isinstance(tool_output, WikiArticle):
            # This means the full_article_tool was used
            print("Full Article:")
            print(f"Title: {tool_output.title}")
            print(f"Content: {tool_output.content[:500]}...")  # Print the first 500 characters
            print(f"URL: {tool_output.url}")
else:
    print("No tool output was returned.")


Tool used: wikipedia_full_article
Full Article:
Title: Eiffel Tower
Content: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists a...
URL: https://en.wikipedia.org/wiki/Eiffel_Tower
